## Canvas Map

Creates an hexagon map as a base for all research and development

Uber hexagon project:
https://nbviewer.jupyter.org/github/uber/h3-py-notebooks/blob/master/H3%20API%20examples%20on%20Urban%20Analytics.ipynb

In [1]:
%%sh
cat <<EOF > requirements.txt
h3==3.1.0
pandas>=0.23
geojson==2.4.1
folium==0.7.0
seaborn==0.9.0
area==1.1.1
statsmodels==0.9.0
shapely
descartes
geopandas
EOF

In [2]:
%%sh
pip3 install -r requirements.txt

In [3]:
%%sh
pip3 install descartes

In [2]:
import json
import pandas as pd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from shapely.geometry import Point, Polygon
import geopandas as gpd
from folium import Map, Marker, GeoJson

In [3]:
coordenadas = pd.read_csv('preprocess/coordenadas.csv')
coordenadas.head(3)

,localidad,latitud,longitud
0,usaquen,4.686949,-74.057169
1,usaquen,4.686786,-74.056385
2,usaquen,4.686443,-74.054938


In [4]:
#Delete Sumapaz, usme, Ciudad Bolivar y san cristobal because are big locations without much interest population
coordenadas = coordenadas[coordenadas.localidad != 'sumapaz']
coordenadas = coordenadas[coordenadas.localidad != 'ciudad bolivar']
coordenadas = coordenadas[coordenadas.localidad != 'usme']
coordenadas

,localidad,latitud,longitud
0,usaquen,4.686949,-74.057169
1,usaquen,4.686786,-74.056385
2,usaquen,4.686443,-74.054938
3,usaquen,4.686387,-74.054670
4,usaquen,4.686233,-74.053883
...,...,...,...
7251,rafael uribe,4.586374,-74.129284
7252,rafael uribe,4.586160,-74.129764
7253,rafael uribe,4.586068,-74.130016
7254,rafael uribe,4.585993,-74.130325


In [5]:
#Transform pandas coordenadas to geopandas points
geo_coord = gpd.GeoDataFrame(
    coordenadas, geometry=gpd.points_from_xy(coordenadas.longitud, coordenadas.latitud))
geo_coord.head(2)

,localidad,latitud,longitud,geometry
0,usaquen,4.686949,-74.057169,POINT (-74.05717 4.68695)
1,usaquen,4.686786,-74.056385,POINT (-74.05638 4.68679)


In [6]:
#Transform geopandas point to localidad polygons
geo_coord = geo_coord.groupby('localidad')['geometry'].apply(lambda x: Polygon(x.tolist())).reset_index()
geo_coord.head(2)

,localidad,geometry
0,antonio narino,"POLYGON ((-74.13088459999999 4.5936364, -74.13..."
1,barrios unidos,"POLYGON ((-74.0933105 4.666218499999999, -74.0..."


In [7]:
#Create an aggregate crokies of bogota city
from shapely.ops import cascaded_union

boundary = gpd.GeoSeries(cascaded_union(geo_coord['geometry']))
bog_bound = boundary.boundary.to_json()

In [8]:
map_bogota = Map(location= [4.6736789,-74.0400262], zoom_start=10, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
GeoJson(
        bog_bound,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'green',
            'weight': 3,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(map_bogota)
map_bogota.save('preprocess/map_bogota.html')
map_bogota

### Fill Bogota with hexagons

In [9]:
#Before filling map with hexagons, coordinates must be inverted in order to H3 library to work
geo_coord_rev = gpd.GeoDataFrame(
    coordenadas, geometry=gpd.points_from_xy(coordenadas.latitud, coordenadas.longitud))
geo_coord_rev = geo_coord_rev.groupby('localidad')['geometry'].apply(lambda x: Polygon(x.tolist())).reset_index()
boundary_rev = gpd.GeoSeries(cascaded_union(geo_coord_rev['geometry']))
bog_bound_rev = boundary_rev.boundary.to_json()
geoJson_rev = json.loads(bog_bound_rev)
geoJson_rev['features'][0]['geometry']['type'] = 'Polygon'

In [10]:
from h3 import h3

set_hexagons = h3.polyfill(geo_json = geoJson_rev['features'][0]['geometry'], res = 8)
print(type(set_hexagons))
list_hexagons = list(set_hexagons)
print("the city have around ", len(list_hexagons), "hexagons at resolution 8")

<class 'set'>
the city have around  594 hexagons at resolution 8


### Get latitud and logitud of a single hexagon

In [11]:
def reverse_lat_lon(hex_coords):
    geom_hex = []
    for lat_lon in hex_coords:
        geom_hex.append([lat_lon[1],lat_lon[0]])
        
    return geom_hex

In [12]:
one_hex_of_fill =  list_hexagons[0]
one_hex_of_fill_coords_latlon = h3.h3_to_geo_boundary(h3_address=one_hex_of_fill,geo_json=False)
one_hex_of_fill_coords_lonlat = reverse_lat_lon(hex_coords = one_hex_of_fill_coords_latlon)

print(one_hex_of_fill)
print(one_hex_of_fill_coords_latlon)
print(one_hex_of_fill_coords_lonlat)

8866e09233fffff
[[4.630610148193411, -74.09691662049886], [4.6279286618470366, -74.10093468770532], [4.623076788450254, -74.10041718737125], [4.620906543346122, -74.09588151665287], [4.623588217854022, -74.09186352676352], [4.628439949295388, -74.09238113026652]]
[[-74.09691662049886, 4.630610148193411], [-74.10093468770532, 4.6279286618470366], [-74.10041718737125, 4.623076788450254], [-74.09588151665287, 4.620906543346122], [-74.09186352676352, 4.623588217854022], [-74.09238113026652, 4.628439949295388]]


### Draw Bogota's Map with hexagons

In [13]:
from geojson.feature import *
import folium

def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already, along with the columns hex_id and value '''
    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

In [14]:
df_fill_hex = pd.DataFrame({"hex_id": list_hexagons})
df_fill_hex["value"] = 0
df_fill_hex['geometry'] = df_fill_hex.hex_id.apply(lambda x: 
                                                       {    "type" : "Polygon",
                                                             "coordinates": 
                                                   [reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=x,geo_json=False))]
                                                        }
                                                    )
geojson_hx = hexagons_dataframe_to_geojson(df_fill_hex)

In [15]:
GeoJson(
        geojson_hx,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'blue',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons" 
    ).add_to(map_bogota)

folium.map.LayerControl('bottomright', collapsed=False).add_to(map_bogota)
map_bogota.save('preprocess/map_bogota.html')
map_bogota

In [19]:
set_hexagons_b = h3.polyfill(geo_json = geoJson_rev['features'][0]['geometry'], res = 7, geo_json_conformant = False)
list_hexagons_b = list(set_hexagons_b)
df_fill_hex_b = pd.DataFrame({"hex_id": list_hexagons_b})
df_fill_hex_b['value'] = 0
df_fill_hex_b['geometry'] = df_fill_hex_b.hex_id.apply(lambda x: 
                                                       {    "type" : "Polygon",
                                                             "coordinates": 
                                                   [reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=x,geo_json=False))]
                                                        }
                                                    )
geojson_hx_b = hexagons_dataframe_to_geojson(df_fill_hex_b)
print("The map with resolution 7 hexagons have " + str(len(list_hexagons_b)))

The map with resolution 7 hexagons have 88


In [20]:
GeoJson(
        geojson_hx_b,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons_b" 
    ).add_to(map_bogota)

folium.map.LayerControl('bottomright', collapsed=False).add_to(map_bogota)
map_bogota.save('preprocess/map_bogota.html')
map_bogota

In [21]:
mapa_bogota = Map(location= [4.6736789,-74.0400262], zoom_start=10, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
GeoJson(
        bog_bound,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'green',
            'weight': 3,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(mapa_bogota)

GeoJson(
        geojson_hx,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'blue',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons" 
    ).add_to(mapa_bogota)

GeoJson(
        geojson_hx_b,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons_b" 
    ).add_to(mapa_bogota)
mapa_bogota

In [51]:
#Output json files for bogotas map, hexagons resolution 7 and 8
with open('source/map_bogota.json', 'w') as f:
    json.dump(bog_bound, f)

In [52]:
with open('source/map_bogota_hex8.json', 'w') as f:
    json.dump(geojson_hx, f)

In [53]:
with open('source/map_bogota_hex7.json', 'w') as f:
    json.dump(geojson_hx_b, f)

['8866e429b9fffff',
 '8866e09343fffff',
 '8866e091ddfffff',
 '8866e092a9fffff',
 '8866e0934bfffff',
 '8866e428d3fffff',
 '8866e09165fffff',
 '8866e428b7fffff',
 '8866e092adfffff',
 '8866e42841fffff',
 '8866e092b7fffff',
 '8866e09169fffff',
 '8866e0926dfffff',
 '8866e09321fffff',
 '8866e09205fffff',
 '8866e42d39fffff',
 '8866e4283dfffff',
 '8866e09051fffff',
 '8866e428abfffff',
 '8866e0924dfffff',
 '8866e09237fffff',
 '8866e42135fffff',
 '8866e42ae5fffff',
 '8866e428a1fffff',
 '8866e0935dfffff',
 '8866e4298bfffff',
 '8866e42f2bfffff',
 '8866e42d63fffff',
 '8866e09235fffff',
 '8866e42f21fffff',
 '8866e46483fffff',
 '8866e4289bfffff',
 '8866e42993fffff',
 '8866e4299dfffff',
 '8866e42d4dfffff',
 '8866e428ddfffff',
 '8866e09089fffff',
 '8866e42a13fffff',
 '8866e09229fffff',
 '8866e090c1fffff',
 '8866e42859fffff',
 '8866e09211fffff',
 '8866e421a5fffff',
 '8866e4649bfffff',
 '8866e09325fffff',
 '8866e42ab7fffff',
 '8866e0903bfffff',
 '8866e09161fffff',
 '8866e42ad1fffff',
 '8866e429dbfffff',
